## Building A Chatbot
Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This code will cover the basics.

In [2]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key



'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'

In [3]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001A366C44E10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001A366F55A50>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")])

AIMessage(content="Hello Krish, it's nice to meet you!\n\nThat's a fascinating role. What kind of AI projects are you working on?  \n\nI'm always eager to learn more about the cutting-edge work being done in the field of AI.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 22, 'total_tokens': 78, 'completion_time': 0.101818182, 'prompt_time': 0.002119458, 'queue_time': 0.018400092, 'total_time': 0.10393764}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-56dca25e-c1ac-49bf-96f6-6771c9a664ce-0', usage_metadata={'input_tokens': 22, 'output_tokens': 56, 'total_tokens': 78})

In [5]:
# one naive way to make conversation history By passing a sequence of messages
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer"),
        AIMessage(content="Hello Krish! It's nice to meet you. \n\nAs a Chief AI Engineer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You are Krish, and you are a Chief AI Engineer.  \n\nIs there anything else you'd like me to know about you or your work?  I'm here to help! 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 97, 'total_tokens': 141, 'completion_time': 0.08, 'prompt_time': 0.005808324, 'queue_time': 0.020129624999999998, 'total_time': 0.085808324}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-524ddc0f-2f1a-468e-a6d7-ecc1cc418ec2-0', usage_metadata={'input_tokens': 97, 'output_tokens': 44, 'total_tokens': 141})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input.

1.	ChatMessageHistory is a simple in-memory (temporary) for chat history
2.	BaseChatMessageHistory is an abstract base class (ABC) that defines the structure for chat message history 
3. RunnableWithMessageHistory It’s task is to add memory to runnables to create stateful applications  

###### A function called get_session_history is created to manage chat history for different sessions.
- Each session is identified by a unique session ID and the chat history is stored in a dictionary (store) where the key is the session ID.
- When the session ID is changed, the chatbot no longer remembers the previous conversation, highlighting the importance of session management.

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history) # model + history

##### This is how BaseChatMessageHistory can read session ID so format is important

In [8]:
config={"configurable":{"session_id":"chat1"}}

In [9]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")],
    config=config
)

In [10]:
response.content

"Hello Krish! It's nice to meet you.\n\nThat's a fascinating title! As a large language model, I'm always interested in learning more about how people are using AI in innovative ways.\n\nWhat kind of projects are you working on as Chief AI Engineer?  I'd love to hear more about your work.\n"

In [11]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Krish.  \n\nYou told me at the beginning of our conversation! 😊  \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 108, 'total_tokens': 130, 'completion_time': 0.04, 'prompt_time': 0.005508221, 'queue_time': 0.02109535, 'total_time': 0.045508221}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-f8059f1b-2353-4f73-a221-3d3b44bda605-0', usage_metadata={'input_tokens': 108, 'output_tokens': 22, 'total_tokens': 130})

In [12]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to know! 😊\n"

In [13]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John! It's nice to meet you.  \n\nWhat can I do for you today?\n"

In [14]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'You said your name is John!  \n\nIs there anything else I can help you with? 😊\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In above case, the raw user input is just a message (Human message), which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking human messages as input). Next, we'll add in more input besides just the messages.

##### MessagePlaceholder
MessagesPlaceholder is a placeholder that tells LangChain where to insert past chat history when formatting a prompt.

In [15]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [16]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")]}) # messages var is replaced in template with value

AIMessage(content="Hi Krish! 👋 \n\nIt's nice to meet you.  \n\nI'm happy to answer any questions you have to the best of my ability.  What's on your mind? 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 31, 'total_tokens': 77, 'completion_time': 0.083636364, 'prompt_time': 0.002329136, 'queue_time': 0.018418562, 'total_time': 0.0859655}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-d70f3419-54b9-4dba-bf62-37addbbde740-0', usage_metadata={'input_tokens': 31, 'output_tokens': 46, 'total_tokens': 77})

##### If there is only one replaceble parameter like messages then no need to explicitly declare it while defining runnable parameter.

In [17]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [18]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Krish")],
    config=config
)

response

AIMessage(content="Hi Krish! It's nice to meet you. 😊\n\nWhat can I do for you today? Ask me anything! I'll do my best to help.  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 31, 'total_tokens': 70, 'completion_time': 0.070909091, 'prompt_time': 0.002314823, 'queue_time': 0.019132675, 'total_time': 0.073223914}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-20ccd55a-943d-44ca-8947-e2439a599a06-0', usage_metadata={'input_tokens': 31, 'output_tokens': 39, 'total_tokens': 70})

In [19]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Krish.  I remember! 😊 \n\nIs there anything else I can help you with?\n'

##### To increase complexity now we add 2 replaceable parameter in input
Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [20]:
## Add more complexity
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [21]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Krish")],"language":"Hindi"})
response.content

'नमस्ते कृष्ण! \n\nमुझे आपसे मिलकर खुशी हो रही है। आप कैसे हैं?  \n\nक्या मैं आपके लिए कुछ कर सकता हूँ? 😊\n'

In [22]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages" #we need to specify the correct key to use to save the chat history.
)

In [23]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Krish")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते कृष्ण! \n\nमुझे बहुत अच्छा लग रहा है कि आप मुझसे बात करना चाहते हैं। आप मुझसे क्या पूछना चाहते हैं? मैं आपकी पूरी मदद करने के लिए यहाँ हूँ। 😊 \n'

In [25]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [26]:
response.content

'तुम्हारा नाम कृष्ण है।  \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
- 'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages etc.

In [27]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last", # trim old memory first
    token_counter=model, # use LLM model for counter
    include_system=True, # include system msg
    allow_partial=False, # msg will not come partially if it is exceeding limit
    start_on="human"  # start trimming from human msg if needed.
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

a:\Study\Data Science\Projects on Langchain\Memory component implementation\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
a:\Study\Data Science\Projects on Langchain\Memory component implementation\.venv\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\aksha\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Deve

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

- itemgetter("messages"): Extracts the "messages" key from an input dictionary.
- RunnablePassthrough: A basic runnable that simply passes data through without modifications.

#### Using Chains for Automatic Trimming

In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

# temgetter("messages") extracts the "messages" key from the input dictionary.
# | trimmer then trims the messages
# The trimmed messages are assigned back to the "messages" key. with .assign

#New chain with trimmer
chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content # reason: TRIMMED

"As an AI, I don't have access to your personal information, including your ice cream preferences. \n\nWhat's your favorite flavor? 😊🍦\n"

In [31]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what\'s 2 + 2".\n'

In [32]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain, # CHAIN WITH TRIMMER
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [38]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="What ice cream do i like?")],
        "language": "English",
    },
    config=config,
)

response.content # out of context due to trim

"As a large language model, I don't have access to your personal information, so I don't know what your favorite ice cream flavor is.  \n\nWhat's your favorite flavor? Maybe I can tell you some fun facts about it! 🍦😊\n"

In [36]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

'You asked "whats 2 + 2"  😄 \n\n\n'